# `MODULO "INGESTOR"`

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Bisogna decidere quali tipi di file “accettare” per l’ingestion. Attualmente abbiamo optato per <strong>.csv</strong>, <strong>.txt</strong> e <strong>.xls</strong>, in futuro si potrebbe ragionare anche sull'accettare <strong>.xml</strong> e <strong>.json</strong>.</p>
</div>

In [808]:
# Definiamo il file per l'ingestion, in futuro sarà probabilmente un URL
fact_table = './Fact_Tables/prova.csv'

# `Operazioni preliminari`
### ➔ Campionamento

La prima considerazione da effettuare è se eseguire o meno il campionamento, soprattutto
nel caso di file enormi.

A questo proposito sarebbe, però, prima opportuno eseguire un po’ di **test di performance** su
file grandi. In caso di performance scadenti bisogna individuare un `metodo di
campionamento efficiente`, randomico, che limiti il più possibile eventuali bias ed errori.

Bisogna anche considerare che per l’individuazione dei tipi di una colonna, il
campionamento può essere molto meno numeroso rispetto a quello necessario ad effettuare
le distinct sui campi, per cui nessun valore deve esser perso.

In [809]:
# 1) Algoritmo per il campionamento (se necessario).
#    ...

# 2) Potrebbe bastare dividere il file in chenks e parallelizzare il lavoro (vedi libreria nella nota sotto).
#    ...

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">
La libreria <a class="reference external" href="https://github.com/Wittline/csv-schema-inference/blob/main/csv_schema_inference/csv_schema_inference.py">csv-schema-inference </a> è interessante, difficilmente integrabile nel nostro algoritmo, ma da cui si potrebbe prendere spunto per implementare un sistema di <strong>calcolo parallelo</strong> per la gestione di files di dimensioni molto grandi.
</p>
</div>

### ➔ Determinare separatori e delimitatori
Oltre alla `codifica` (Ascii, Unicode, UTF-8, ect.) e alla `convenzione di fine record`, che
normalmente sono identificati automaticamente dal tool che legge il file, vanno identificati anche i
`separatori di campo` (virgola, punto e virgola, tab, spazio, etc.) e i `delimitatori di stringa` (opzionali,
tipicamente “”). Anche unaidentificazione automatica delle `righe di intestazione` sarebbe utile.

In [810]:
import chardet

"""
get_encoding(<filename>): funzione che identifica l'encoding di un file .csv
params:
    <filename>: String, il file .csv di cui vogliamo sapere l'encoding
output:
    String, stringa contenente il nome dell'encoding
"""

def get_encoding(filename):
    with open(filename, 'rb') as f:
        encoding = chardet.detect(f.read(10000))
        f.close()
    return encoding.get('encoding')

In [811]:
# Test della funzione get_encoding
get_encoding(fact_table)

'UTF-8-SIG'

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last"> La funzione <strong>get_encoding</strong> mi restituisce l'encoding presente nel file .csv con una certa attendibilità [0,1]. L'encoding mi risolve anche eventuali unità di misura, e.g. $, £, ...) che costituiscono una informazione da estrarre successivamente.
Per maggiori informazioni sulla libreria <a class="reference external" href="https://github.com/chardet/chardet">Chardet</a>.
</p>
<p> Sebbene sembrerebbe necessaria una migliore gestione della selezione delle righe in modo random, è provato empiricamente che questa sia una soluzione valida.
</div>

In [812]:
# La libreria CleverCSV usa un modello di ML per stabilire delimiter, quotechar and escaperchar del file .csv.
# La performance di identificazione è nettamente superioreis a quella di altre librerie CSV di python.
import clevercsv

"""
get_clever_delimiter(<filename>): funzione che identifica delimiter, quotechar and escaperchar di un file .csv
params:
    <filename>: String, il file .csv
output:
    tuple, la tupla (<delimiter>, <quotechar>, <escapechar>)
"""

def get_clever_delimiter(filename):
    with open(filename, newline='') as f:
        dialect = clevercsv.Sniffer().sniff(f.read(10000))
        f.close()
    return dialect.delimiter, dialect.quotechar, dialect.escapechar

In [813]:
# Test della funzione get_clever_delimiter
get_clever_delimiter(fact_table)

(';', '', '')

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Per maggiori informazioni sulla libreria CleverCSV, far riferimento alla relativa <a class="reference external" href="https://github.com/alan-turing-institute/CleverCSV">documentazione</a>.
   CleverCSV può essere utilizzato tipo la sostituzione drop-in del <a class="reference external" href="https://docs.python.org/3/library/csv.html#csv.Sniffer"> modulo CVS di python</a>
</p>
</div>

In [814]:
import csv

"""
get_header(<filename>): funzione che stabilisce se il file .csv possiede una header
params:
    <filename>: String, il file .csv
output:
    bool, se True, il file .csv possiede una header
"""

def get_header(filename):
    with open(filename) as f:
        header = csv.Sniffer().has_header(f.read(10000))
        f.close()
    return header

In [815]:
# Test della funzione get_he.ader
get_header(fact_table)

True

Inprecedenza ho definito tutte le funzioni necessarie per `aprire correttamente il file csv.` Tali funzioni vengono ora impiegate nel codice sottostante per aprire e `salvare la fact table di input sottoforma di Pandas Dataframe` in una variabile **df_fact_table**.

In [816]:
import pandas as pd
import numpy as np


# Apro il file che voglio importare come un dataFrame e lo salvo nella variabile df_fact_table
try:
    # Nel caso in cui il file abbia formato .xlsx
    if '.xlsx' in fact_table:
        df_fact_table = pd.read_excel(fact_table)
        
    # In tutti gli altri casi (quindi .csv, .txt, etc.)
    else:
        encoding = get_encoding(fact_table)
        header = get_header(fact_table)
        delimiter, quotechar, escapechar = get_clever_delimiter(fact_table)
        
        if header == False:
            header = None
        else:
            header = 'infer'
            
        if quotechar == '':
            quotechar = '"'
            
        if escapechar == '':
            escapechar = None
        
        if encoding is not None:
            df_fact_table = pd.read_csv(fact_table, header = header, sep = delimiter, quotechar = quotechar, encoding = encoding, low_memory = False, on_bad_lines = 'skip', encoding_errors = 'ignore')
 
            # Costruisco la header con dei placeholders, nel caso in cui non sia presente
            if header == None:
                n_columns = len(list(df_fact_table))
                positions = range(0, n_columns)

                dims = {}
                for i in positions:
                    dims[i] = 'dim' + str(i)
                    
                df_fact_table = df_fact_table.rename(columns = dims)
                
        else:
            print(f'Unknown encoding {fact_table}')

# Raccolgo qui eventuali eccezioni
except Exception as e:
    print(e)

In [817]:
df_fact_table

,data,ora,filiale,CAP,regione,codice prodotto,prodotto,classe prodotto,famiglia prodotto,operatore,operazione,unità,importo,COD
0,15/12/2021,08:30,Torino,10121,Piemonte,123,Kiwi Phone 12 Pro,Smartphone,Personal Devices,Camillo Benso,Vendita,2.0,"1.980,00 €",123
1,15/12/2021,09:30,Genova,16167,Liguria,123,Kiwi Phone 12 Pro,Smartphone,Personal Devices,Gino Paoli,Vendita,1.0,"975,59 €",123
2,15/12/2021,10:30,Potenza,85100,Basilicata,456,Bob Kitchen Robot,Elettrodomestici,Home Devices,Alberto Lupo,Vendita,1.0,"2.547,00 €",456
3,15/12/2021,11:30,Verona,37125,Veneto,456,Bob Kitchen Robot,Elettrodomestici,Home Devices,Billy Ballo,Vendita,1.0,"2.869,79 €",456
4,16/12/2021,12:30,Torino,10121,Piemonte,789,Carl Vacuum Cleaner,Elettrodomestici,Home Devices,Camillo Benso,Riparazione,NaN,NaN,789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,07/05/2022,21:45,Torino,10121,Piemonte,105,Jack Earphones 4,Audio,Personal Devices,Vittorio Emanuele,Vendita,1.0,"92,95 €",105
196,08/05/2022,10:30,Roma,135,Lazio,911,Mango Pad 8 Mini,Tablet,Personal Devices,Anco Marzio,Vendita,2.0,"1.211,60 €",911
197,08/05/2022,11:30,Torino,10121,Piemonte,456,Bob Kitchen Robot,Elettrodomestici,Home Devices,Vittorio Emanuele,Vendita,1.0,"2.499,90 €",456
198,09/05/2022,12:30,Asti,14100,Piemonte,774,Mango Pad 8 Pro Max,Tablet,Personal Devices,Pino Loricato,Vendita,1.0,"1.029,60 €",774


<div class="admonition warning alert alert-danger">
<p class="first admonition-title" style="font-weight: bold;">Attenzione</p>
<p class="last">I parametri <strong>on_bad_lines = 'skip'</strong> and <strong>encoding_errors = 'ignore'</strong> andrebbero in futuro trattati opportunamente. Al momento sono forzati.</p>
</div>

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Controllo se la header della fact table è presente, se non è presente assegno un placeholder (dim1, dim2, ...) come label.</p>
</div>

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">La variabile globale <strong>df_fact_table</strong> è molto importante e verrà utilizzata e modificata nel corso del codice che segue, per diventare parte dell'output dell'algoritmo Ingestor.</p>
</div>

###### Ispezione visiva della "fact table originale".
df_fact_table.head(20)

# `Operazioni di manipolazione`
`Ho deciso di eseguire prima la distinct` e operare sulle tabelle esterne ottenute per rimuovere le colonne float e string.

### ➔ Esecuzione delle “Distinct”
Questa operazione serve ad ottenere un elenco di tutte le occorrenze univoche presenti in una colonna.
`Le distinct di tutte le colonne vengono eseguite in una sola “passata” della fact table o del campione.`
L’output di questo step è un `dizionario` salvato nella variabile **tabelle_esterne**, il quale ha come valori le “tabelle esterne” e come chiavi corrispondenti i nomi delle colonne (dimensioni).

Notare come il numero di tabelle esterne generate è pari al numero delle colonne della fact table.

In [818]:
# Dizionario contenente le tabelle esterne a cui sono stati tolti i valori nulli ed è stata applicata una DISTINCT
tabelle_esterne = {}

for column in df_fact_table:
    
    tabelle_esterne[column] = pd.DataFrame(df_fact_table[column]).dropna().drop_duplicates().astype(str).reset_index(drop=True)
    
    # Scarto già tutte le colonne che hanno solamente un valore (e.g. contenti solo NaN o separatori inutili)
    #if len(tabelle_esterne[column].index) <= 1:
    #    tabelle_esterne.pop(column, None)
        

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Basterebbe porre qui sopra la <strong>condizione <=2 per escludere anche tutte le dimensioni booleane</strong> (com'è stato suggerito da Roberto).</p>
</div>

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">il dizionario variabile globale <strong>tabelle_esterne</strong> è molto importante e verrà utilizzata e modificata nel corso del codice che segue, per diventare parte dell'output dell'algoritmo Ingestor.</p>
</div>

In [819]:
# Ispezione visiva delle chiavi del dizionario
tabelle_esterne.keys()

dict_keys(['data', 'ora', 'filiale', 'CAP', 'regione', 'codice prodotto', 'prodotto', 'classe prodotto', 'famiglia prodotto', 'operatore', 'operazione', 'unità', 'importo', 'COD'])

In [820]:
# Ispezione visiva di una tabellla esterna salvata nel dizionario
#tabelle_esterne['CAP']

### ➔ Creo la tabella che tiene "traccia" della informazioni sulla fact table originale 
`Questa tabella tiene traccia delle informazioni relative a ciascuna colonna (dimensione)`. Essa è utile in vari step dell'algoritmo.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">L’<strong>output</strong> delle operazioni preliminari dev’essere, oltre che al dizionario contenente le <strong>tabelle_esterne</strong>, un DataFrame <strong>tabella_traccia</strong> che tracci le informazioni relative alle colonne della fact table originale (e.g. il nome della colonna, il tipo, il formato, etc.). Ogni riga di tale DataFrame contiene dunque le informazioni di una certa colonna presente nella fact table originale.</p>

Notare come la posizione di una colonna nella fact table originaria corrisponde all'indice della riga in tabella_traccia.
</div>

In [821]:
# Costruisco la "tabella traccia" che contiene le informazioni di ciascuna colonna.                
info_colonne = {'dimensione': tabelle_esterne.keys(), 'unità': np.NaN, 'tipo': np.NaN, 'formato': np.NaN, 'lingua': np.NaN, 'relazione': np.NaN}
tabella_traccia = pd.DataFrame(info_colonne)

In [822]:
# Ispezione visiva della "tabella traccia"
tabella_traccia

,dimensione,unità,tipo,formato,lingua,relazione
0,data,NaN,NaN,NaN,NaN,NaN
1,ora,NaN,NaN,NaN,NaN,NaN
2,filiale,NaN,NaN,NaN,NaN,NaN
3,CAP,NaN,NaN,NaN,NaN,NaN
4,regione,NaN,NaN,NaN,NaN,NaN
5,codice prodotto,NaN,NaN,NaN,NaN,NaN
6,prodotto,NaN,NaN,NaN,NaN,NaN
7,classe prodotto,NaN,NaN,NaN,NaN,NaN
8,famiglia prodotto,NaN,NaN,NaN,NaN,NaN
9,operatore,NaN,NaN,NaN,NaN,NaN


### ➔ Individuazione dei tipi delle colonne
`Questa operazione viene eseguita sulle tabelle esterne appena create`, poiché considerato un subset sufficiente della fact table originaria.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Come <strong>output</strong> di questo step andiamo ad <strong>aggiungere il tipo e l'eventuale formato</strong> alla "tabella traccia".</p>
<p>Dopodichè cancelliamo dal dizionario "tabelle_esterne" le tabelle esterne di tipo Float, Stringa e quelle di tipo Integer non categoriche.</p>
</div>

### 1) Time
`Una colonna è di tipo Time se rappresenta un timestamp (giorno, mese, anno, ora, minuto,
secondo) in un qualsiasi formato valido e se ogni sottocomponente di un timestamp è
riconoscibile come tale`. Questo permette di identificare come Time espressioni limitate tipo:
‘05-2022’ o semplicemente ‘Oct’, ‘2022’ o ’18:30’.
Può essere utile vedere la reduced precision di ISO 8601 o anche le truncated expression (ora deprecate ma la cosa non è
rilevante per i nostri fini).
Bisogna considerare il problema derivante dall’enorme varietà di casi.
YYYY/MM/DD hh:mm:ss (ad esempio possono essere combinati in n modi)
Bisogna trovare una libreria che faccia bene il lavoro, oppure sviluppare un metodo ex-novo
cominciando a coprire una casistica limitata ma significativa di formati.
In questo senso va approfondito l’npm “any-date-parser”.
L’individuazione di una colonna come tipo Time deve fornire come output la data in formato
standard scelto da noi (DD/MM/YYYY) e deve tenere traccia del formato originale.

<div class="admonition warning alert alert-danger">
<p class="first admonition-title" style="font-weight: bold;">Attenzione</p>
<p class="last">Sviluppare correttamente le funzioni e gestire correttamamente errori ed eccezioni. Valutare un preprocessing che traduca in inglese i nomi dei mesi.</p>
</div>

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">
In caso avessimo date complete, e.g. gg/mm/yyyy hh:mm; potremmo pensare di gerarchizzare in modo automiatico i dati inseriti andando a definire le colonne anno, mese e ora in <strong>gerarchia</strong> ammesso che abbia senso. Al contrario si può pensare di di aggregare date e tempo in un unico timestamp. Dipende se il la dimensione temporale è da considerarsi lineare o ciclica
</p>
</div>

In [823]:
from datetime import datetime
import dateutil.parser as dateutilParser


# Definisco la classe CustomParser per customizzare il parser di dateutil, che altrimenti è troppo generico per i nostri scopi.
class CustomParser:

    def __init__(self, year=0, month=0, day=0, hour=0, minute=0, second=0, microsecond=0, default=None):
        self._year = year
        self._month = month
        self._day = day
        self._hour = hour
        self._minute = minute
        self._second = second
        self._microsecond = microsecond

        if default is None:
            default = datetime.now() # I microsecond non li considero mai

        self.year = default.year
        self.month = default.month
        self.day = default.day
        self.hour = default.hour
        self.minute = default.minute
        self.second = default.second
        self.microsecond = default.microsecond
        self.default = default

    # Le seguenti proprietà dell'oggetto
    @property
    def has_year(self):
        return self._year > 1900 and self._year < 2030

    @property
    def has_month(self):
        return self._month != 0

    @property
    def has_day(self):
        return self._day != 0
    
    @property
    def has_hour(self):
        return self._hour != 0
    
    @property
    def has_minute(self):
        return self._minute != 0
    
    @property
    def has_second(self):
        return self._second != 0

    def set_year(self, year):
        self._year = year
    
    def set_month(self, month):
        self._month = month

    def set_day(self, day):
        self._day = day
    
    def set_hour(self, hour):
        self._hour = hour
    
    def set_minute(self, minute):
        self._minute = minute
    
    def set_second(self, second):
        self._second = second

    @property
    def todatetime(self):
        res = {
            attr: value
            for attr, value in [
                ("year", self._year),
                ("month", self._month),
                ("day", self._day),
                ("hour", self._hour),
                ("minute", self._minute),
                ("second", self._second),
                ("microsecond", self._microsecond),
            ] if value
        }
        return self.default.replace(**res)

    def replace(self, **result):
        return CustomParser(**result, default=self.default)

    def __repr__(self):
        return "%s(%d, %d, %d, %d, %d, %d, %d)" % (
            self.__class__.__qualname__,
            self._year,
            self._month,
            self._day,
            self._hour,
            self._minute,
            self._second,
            self._microsecond
        )  

In [824]:
# Inizializzo l'oggetto del parser customizzato, il quale viene utilizzato nelle funzioni definite per il parsing
sentinel = CustomParser()

def is_year(string, sentinel=sentinel):
    """
    is_year(<string>): funzione che stabilisce se una stringa può essere interpretata come un anno.
    params:
        <string>: String, la stringa di cui voglio sapere se è un anno
    output:
        Bool
    """
    try: 
        date = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return (date.has_year and not date.has_month)
    
    except ValueError:
        return False

    
def is_month(string, sentinel=sentinel):
    """
    is_month(<string>): funzione che stabilisce se una stringa può essere interpretata come un mese.
    params:
        <string>: String, la stringa di cui voglio sapere se è un mese
    output:
        Bool
    """
    try: 
        date = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return (date.has_month and not date.has_year)

    except ValueError:
        return False


def is_date(string, sentinel=sentinel):     
    """
    is_date(<string>): funzione che stabilisce se una stringa può essere interpretata come una data.
    params:
        <string>: String, la stringa di cui voglio sapere se è una data
    output:
        Bool
    """
    try: 
        date = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return (date.has_month and date.has_year and not date.has_hour)

    except ValueError:
        return False
    

def is_time(string, sentinel=sentinel):
    """
    is_time(<string>): funzione che stabilisce se una stringa può essere interpretata come un tempo.
    params:
        <string>: String, la stringa di cui voglio sapere se è un tempo
    output:
        Bool
    """
    try: 
        time = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return (time.has_hour and not time.has_year)

    except ValueError:
        return False
    
    
def is_timestamp(string, sentinel=sentinel):
    """
    is_time(<string>): funzione che stabilisce se una stringa può essere interpretata come un timestamp.
    params:
        <string>: String, la stringa di cui voglio sapere se è un timestamp
    output:
        Bool
    """
    try: 
        timestamp = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return (timestamp.has_hour and timestamp.has_year)

    except ValueError:
        return False


def standard_date(string, sentinel=sentinel):
    """    
    standard_date(<string>): funzione che converte una data in un formato standard scelto
    params:
        <string>: String, la data che voglio convertire in formato standard
    output:
        datetime.date, data in formato standard aaaa-mm-gg
    """
    try: 
        date = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return date.todatetime.date()

    except ValueError:
        return None


def standard_time(string, sentinel=sentinel):
    """    
    standard_time(<string>): funzione che converte una un'ora in un formato standard scelto
    params:
        <string>: String, la data che voglio convertire in formato standard
    output:
        datetime.time, ora in formato standard hh-mm-ss
    """
    try: 
        time = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return time.todatetime.time().replace(microsecond=0)

    except ValueError:
        return None
 

"""
____________________________________________________________________________________________________
Funzioni per la conversione nel timestamp luna
"""
    
def standard_timestamp(string, sentinel=sentinel):
    try: 
        time = dateutilParser.parse(string, ignoretz=True, default=sentinel)
        return time
    except ValueError:
        return None
    
    
def addZero(num_):
    if(num_<10):
        return '0' + str(num_)
    else:
        return str(num_)
    
    
def convertToLunaTimestamp(df):
    for idx in range(len(df)):
        pippo = df.loc[idx, 'timestampLuna']
        _ = ''
        _ = _ + str(pippo._year) 
        _ = _ + addZero(pippo._month)
        _ = _ + addZero(pippo._day)
        _ = _ + addZero(pippo._hour)
        _ = _ + addZero(pippo._minute)
        _ = _ + addZero(pippo._second)
        
        df.loc[idx, 'timestampLuna'] = _
        
    return df


def buildLunaTimestamp(df):
    df['timestampLuna'] = standard_timestamp('0')
    
    if ('timestamp' in df.columns):
        df['timestampLuna'] = df['timestamp'].apply(lambda x: standard_timestamp(x))
        return df
        
    if ('date' in df.columns):
        df['timestampLuna'] = df['date'].apply(lambda x: standard_timestamp(x))
        
        if ('time' in df.columns):
            df['time'] = df['time'].apply(lambda x: standard_timestamp(x))
            
            for idx in range(len(df)):
                df.loc[idx, 'timestampLuna'].set_hour(df.loc[idx, 'time']._hour)
                df.loc[idx, 'timestampLuna'].set_minute(df.loc[idx, 'time']._minute)
                df.loc[idx, 'timestampLuna'].set_second(df.loc[idx, 'time']._second)

    else:
        if ('time' in df.columns):
            df['timestampLuna'] = df['time'].apply(lambda x: standard_timestamp(x))
        
        if ('year' in df.columns):
            df['year'] = df['year'].astype(str).apply(lambda x: standard_timestamp(x))
            
            for idx in range(len(df)):
                df.loc[idx, 'timestampLuna'].set_year(df.loc[idx, 'year']._year)
            
        if ('month' in df.columns):
            df['month'] = df['month'].apply(lambda x: standard_timestamp(x))

            for idx in range(len(df)):
                df.loc[idx, 'timestampLuna'].set_month(df.loc[idx, 'month']._month)
        
 
    return df

In [825]:
# Test per le funfioni che parsificano le date
'''
prova1 = is_year('Wed, 31 Jan 2024 14:51:42 GMT')
print(prova1)

prova2 = is_month('october')
print(prova2)

prova3 = is_date('12.22') # Per non confondere i float
print(prova3)


prova4 = is_time('12:11')
print(prova4)
'''
prova4 = standard_timestamp('2012')
print(prova4)

prova5 = standard_timestamp('Wed, 31 Jan 2024')
prova5.set_year(prova4._year)
print(prova5)

CustomParser(2012, 0, 0, 0, 0, 0, 0)
CustomParser(2012, 1, 31, 0, 0, 0, 0)


<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Le migliori librerie per parsificare le date sembrano essere 
<a class="reference external" href="https://github.com/dateutil/dateutil">dateutil</a>, che è un built in module di python, e 
<a class="reference external" href="https://github.com/arrow-py/arrow">arrow</a>, che è una libreria esterna.</p>
<p class="last">Come per altre librerie interessanti, funzionano solo per parole in lingua inglese (e.g. October, Oct, ...) , conviene considerare dunque un preprocessing che faccia una traduzione da qualsiasi lingua all'inglese</p>
</div>

### 2) Float e Integer
`Bisogna distinguere tra colonne integer e colonne float, ossia tra dimensioni e misure.` Questa operazione viene solo parzialmente eseguita sfruttando l’abilità dei parser di riconoscere formattazioni numeriche valide. Bisogna, tuttavia, prestare attenzione alla convenzione standard utilizzata nella fact table. `Vanno eliminati anche eventuali suffissi e prefissi` (ad esempio simboli, $, £, €, ecc.).
Una volta identificate colonne float ed integer, `le float non verranno più utilizzate ai fini dell’analisi` in quanto rappresentano quasi sicuramente le misure.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last"> I casi che la funzione <strong>parseNumber</strong> riesce a parsificare sono:
    <ul>
        <li> parseNumber("a 125,00 €") >>> 125 </li>
        <li> parseNumber("100.000,000") >>> 100000 </li>
        <li> parseNumber("100,000,000") >>> 100000000 </li>
        <li> parseNumber("100 000 000") >>> 100000000 </li>
        <li> parseNumber("100.001 001") >>> 100.001 </li>
        <li> parseNumber("100 000,000") >>> 100000</li>
        <li> parseNumber("\$.3") >>> 0.3 </li>
        <li> parseNumber(".003") >>> 0.003 </li>
        <li> parseNumber(".003 55") >>> 0.003 </li>
        <li> parseNumber("1.190,00 €") >>> 1190 </li>
        <li> parseNumber("1190,00 €") >>> 1190 </li>
        <li> parseNumber("1,190.00 €") >>> 1190 </li>
        <li> parseNumber("\$1190.00") >>> 1190 </li>
        <li> parseNumber("\$1 190.99") >>> 1190.99 </li>
        <li> parseNumber("\$-1 190.99") >>> -1190.99 </li>
        <li> parseNumber("1 000 000.3") >>> 1000000.3 </li>
        <li> parseNumber('-151.744122') >>> -151.744122 </li>
        <li> parseNumber('-1') >>> -1</li>
        <li> parseNumber("1 0002,1.2") >>> 10002.1</li>
        <li> parseNumber(1) >>> 1</li>
        <li> parseNumber(1.1) >>> 1.1</li>
        <li> parseNumber("rrr1,.2o") >>> 1</li>
        <li> parseNumber("rrr1rrr") >>> 1</li>
    </ul>    
</p>
</div>

In [826]:
import re
            

"""
parseNumber(<string>): funzione che parsifica un numero.
params:
    <string>: String, il numero float o integger che sia, che voglio parsificare
output:
    Float/Integer (a seconda se il numero fornito alla funzionesia un float o un integer)
"""

def parseNumber(text):
    try:
        n = float(text)
        
        if n.is_integer():
            return int(n)
        else:
            return n
    except: pass
    return None


"""
parseAnyNumber(<string>): funzione che parsifica un numero in qualsiasi formato.
params:
    <string>: String, il numero float o integger che sia, che voglio parsificare
output:
    Float/Integer (a seconda se il numero fornito alla funzionesia un float o un integer)
"""

def parseAnyNumber(text):

    try:
        # Se la funzione non ha argomento passato
        if text is None:
            return None
        
        # Se l'argomento passato è già parsificato correttamente
        if isinstance(text, int) or isinstance(text, float):
            return text
        
        text = text.strip()
        
        # Se l'argomento passato è una stringa vuota
        if text == "":
            return None
        
        # Cerco il primo "[0-9,. ]+":
        n = re.search("-?[0-9]*([,. ]?[0-9]+)+", text).group(0)
        n = n.strip()
        
        original_text = text.replace(n, ' ').strip()
        
        if not re.match(".*[0-9]+.*", text):
            return None
    
        
        # Applicao un cut per tenere solo i simboli ' ' ',' '.'
        while " " in n and "," in n and "." in n:
            index = max(n.rfind(','), n.rfind(' '), n.rfind('.'))
            n = n[0:index]
        n = n.strip()
                
        # Conto i simboli ' ' ',' '.':
        symbolsCount = 0
        for current in [" ", ",", "."]:
            if current in n:
                symbolsCount += 1
                
        # Se non trovo simboli non faccio nulla
        if symbolsCount == 0:
            pass
        
        # Se trovo un solo simbilo applico i seguenti criteri di selezione
        elif symbolsCount == 1:
            # Se il simbolo è uno spazio, mi limito a rimuoverso
            # TODO: andrebbe magari gestito per il caso dei numeri con migliaia (e.g. "1 000")
            if " " in n:
                n = n.replace(" ", "")
            # Altrimenti il simbolo è "." oppure va rimosso
            else:
                theSymbol = "," if "," in n else "."
                if n.count(theSymbol) > 1:
                    n = n.replace(theSymbol, "")
                else:
                    n = n.replace(theSymbol, ".")
        else:
            # Rimpiazzo i simboli in modo che il simbolo a destra sia '.', e tutti quelli a sinistra siano ''
            rightSymbolIndex = max(n.rfind(','), n.rfind(' '), n.rfind('.'))
            rightSymbol = n[rightSymbolIndex:rightSymbolIndex+1]
            if rightSymbol == " ":
                return parseNumber(n.replace(" ", "_"))
            n = n.replace(rightSymbol, "R")
            leftSymbolIndex = max(n.rfind(','), n.rfind(' '), n.rfind('.'))
            leftSymbol = n[leftSymbolIndex:leftSymbolIndex+1]
            n = n.replace(leftSymbol, "L")
            n = n.replace("L", "")
            n = n.replace("R", ".")
            
        # Faccio un cast del testo in float oppure int
        n = float(n)
        if n.is_integer():
            return int(n)
        else:
            return n
    except: pass
    return None

In [827]:
from quantulum3 import parser as quantulumParser


"""
parserUnit(<string>): funzione che ci dice se la stringa contiene una unità di misura
params:
    <string>: String
output:
    String, se presente, l'unità di misura rilevata, altrimenti None
"""

def parserUnit(text):
    
    quants = quantulumParser.parse(text)

    for items in quants:
        unit = items.unit.name
        if unit != 'dimensionless':
            return unit
        
"""
parserUnit(<string>): funzione che ci dice se la stringa contiene una unità di misura
params:
    <string>: String
output:
    String, se presente, l'unità di misura rilevata, altrimenti None
"""

def parserExtractQuantity(text):
    
    quants = quantulumParser.parse(text)
    for items in quants:
        unit = items.unit.name
        return items.value

        
"""
hasNumbers(<string>): funzione che ci dice se la stringa contiene almeno un numero
params:
    <string>: String
output:
    Bool
"""

def hasNumbers(text):
    return bool(re.search(r'\d', text))


"""
hasLetter(<string>): funzione che ci dice se la stringa contiene almeno una lettera
params:
    <string>: String
output:
    Bool
"""

def hasLetter(text):
    flag_l = False
    if not re.match("^[0-9,.-]*$", text):
        flag_l=True
            
    return flag_l

In [828]:
# Esempi di test per le funzioni di parsing dei float/ integer

unit = parserUnit(" 1 £")
print('unit: ', unit)

quantity = parserExtractQuantity("1")
quantity

unit:  pound sterling


1.0

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">
<a class="reference external" href="https://github.com/nielstron/quantulum3">Quantulum</a>,
è una libreria per l'estrazzione di informazioni di quantità, misure e unità di misura da testo destrutturato. Questa libreria è anche in grado di disambiguare tra unità simili e simboli ed è basato sull'algoritmo k-nearest neighbours.</p>
<p class="last">Come altre librerie interessanti, funziona solo con parole inglesi (e.g. litre, metre, ..).</p>
</div>

### 3) Bool
`Se la tabella esterna ha solo due righe, molto probabilmente è di tipo booleano.` Un ulteriore semplice controllo che si potrebbe fare è di testare se le righe corrispondono a (TRUE, FALSE, 0, 1, YES, NO, ecc.)

In [829]:
"""    
is_bool(<tabella>): funzione che ci dice se la tabella esterna è di tipo Bool
params:
    <tabella>: dataFrame, la tabella esterna
output:
    Bool
"""

def is_bool(tabella):
    if len(tabella) == 2:
        return True
    else:
        return False

In [830]:
# Test della funzione is_bool
col_test = {'test': [True, False]}
tab_test = pd.DataFrame(col_test)

is_bool(tab_test)

True

### 4) Categorica
Per stabilire se una colonna è di tipo categorico bisogna `valutare il rapporto tra il numero di
occorrenze univoche ed il numero di records totali.`

Solitamente, una colonna categorica, presenta un `numero totale di valori univoci molto
inferiore al numero di righe della tabella.`
Per poter interpretare correttamente il rapporto è necessario procedere in modo empirico,
fare alcuni test e stabilire delle soglie.
In questo modo è possibile anche valutare se una colonna di tipo integer è, magari, una
colonna categorica come nel caso, ad esempio, dei CAP che sono degli interi ma a tutti gli
effetti sono dati categorici.

In [831]:
"""
is_categorical_1(<DataFrame>): funzione che stabilisce se una colonna di un DataFrame è categorica
params:
    <DataFrame>: DataFrame, la colonna di un Pandas DataFrame
output:
    Bool
"""

def is_categorical_1(df_column):
    
    likely_cat = 1. * df_column.dropna().nunique() / df_column.dropna().count()
    
    if likely_cat < 0.05: #or some other threshold
        return True
    else:
        return False

In [832]:
"""
is_categorical_2(<DataFrame>): funzione che stabilisce se una colonna di un DataFrame è categorica
params:
    <DataFrame>: DataFrame, la colonna di un Pandas DataFrame
output:
    Bool
"""

def is_categorical_2(df_column):
    
    top_n = 10     
    likely_cat = 1. * df_column.value_counts(normalize=True).head(top_n).sum()
        
    if likely_cat > 0.8: #or some other threshold
        return True
    else:
        return False

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">
    <ul>
        <li><strong>is_categorical_1</strong> ha un trashold del 5%. Una colonna non verrà mai classificata come categorica se possiede meno di 20 righe. Sarebbe dunque meglio adeguare il trashold alle dimensioni del dataFrame.</li>
        <li><strong>is_categorical_1</strong> funziona generalmente meglio di <strong>is_categorical_2</strong>. Inoltre è più computazionalmente efficiente.</li>
        <li><strong>is_categorical_2</strong> è tuttavia meglio se c'è una distribuzione con una coda lunga, ossia quando pochi valori categorici hanno un'alta frequenza e tanti valori categorici hanno una bassa frequenza.</li>
    </ul>
</p>
</div>

In [833]:
#for dim in df_fact_table:
#    print(is_categorical_1(df_fact_table[dim]))

In [834]:
#for dim in df_fact_table:
#    print(is_categorical_2(df_fact_table[dim]))

### 5) Spazio
`Le colonne categoriche vanno ulteriormente testate per stabilire se contengono dati relativi
allo spazio` (es. indirizzo, continenti, paesi, aree, ecc.) sfruttando qualche libreria GIS

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Esempi di librerie leggere per parsificare città, regioni e stati sono
<a class="reference external" href="https://github.com/kaushiksoni10/locationtagger">locationtagger</a> e
<a class="reference external" href="https://github.com/elyase/geotext">geotex</a>.
    Una libreria più completa ma anche più pensate è <a class="reference external" href="https://github.com/somnathrakshit/geograpy3">geography</a>.
</p>
</div>

### 6) Stringa
`I tipi delle colonne vengono identificati per esclusione.` Le colonne di questo tipo, che dovrebbero essere
abbastanza rare, sono di solito di misure di tipo alfanumerico (come ad es. le note o gli stati
non codificati) o irrilevanti per l’analisi (es. il codice di transazione).

### `Algoritmo di identificazione dei tipi`

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">
Si potrebbe pensare di applicare la libreria <a class="reference external" href="https://github.com/nielstron/quantulum3">Quantulum</a> alle headers ed estrarre informazioni sulle unità di misura che potrebbero non essere presenti nelle colonne del dataFrame.
</p>
</div>

In [835]:
#tabelle_esterne

In [836]:
# I modi più veloce per loopare su un dataFrame sono, o di vettorializzazrlo (trasformarlo in un array numpy), o di
# trasformarlo in un dizionario.

for dim in tabelle_esterne:

    df_sample = tabelle_esterne[dim].sample(30, replace=True)
    df_sample_dict = df_sample.to_dict('records')
    

    # 1) Time

    if all(is_year(row[dim]) for row in df_sample_dict):
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'year'
        continue

    if all(is_month(row[dim]) for row in df_sample_dict):
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'month'
        continue
        
    if all(is_time(row[dim]) for row in df_sample_dict):
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'time'
        continue
        
    if all(is_date(row[dim]) for row in df_sample_dict):
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'date'
        continue
        
    if all(is_timestamp(row[dim]) for row in df_sample_dict):
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'timestamp'
        continue
        

    # 2) Integer/Float

    # Se la stringa è un numero facilmente parsificabile
    if all(parseNumber(row[dim]) is not None for row in df_sample_dict):
        if any(isinstance(parseNumber(row[dim]), float) for row in df_sample_dict):
            tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'float'
            continue
        else:
            if is_categorical_2(df_fact_table[dim]):
                tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'integer categorical'
                continue
            else:
                tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'integer'
                continue

    # Se la stringa è un numero non facilmente parsificabile, i.e. di un formato strano o che contiene lettere/simboli
    if any(parseNumber(row[dim]) is None for row in df_sample_dict):
        if all(hasLetter(row[dim]) for row in df_sample_dict):
            if all(parserUnit(row[dim]) is not None for row in df_sample_dict):
                unit = parserUnit(df_sample_dict[0][dim])
                tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'unità'] = unit
                if all(isinstance(parseAnyNumber(row[dim]), int) for row in df_sample_dict):
                    if is_categorical_2(df_fact_table[dim]):
                        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'integer categorical'
                        continue
                    else:
                        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'integer'
                        continue
                elif any(isinstance(parseAnyNumber(row[dim]), float) for row in df_sample_dict):
                    tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'float'
                    continue
        else:
            if all(isinstance(parseAnyNumber(row[dim]), int) for row in df_sample_dict):
                if is_categorical_2(df_fact_table[dim]):
                    tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'integer categorical'
                    continue
                else:
                    tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'integer'
                    continue
            elif any(isinstance(parseAnyNumber(row[dim]), float) for row in df_sample_dict):
                tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'float'
                continue                

                
    # 3) Bool

    if is_bool(df_sample):
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'bool'
        continue

        
    # 4) Categorica

    if is_categorical_1(df_fact_table[dim]) or is_categorical_2(df_fact_table[dim]):
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'categorical'
        continue
        
        
    # 5) Luogo

    # Ancora da implementare...
    
    
    # 6) Stringa
    
    else:
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim, 'tipo'] = 'string'

In [837]:
# Ispezione visiva della tabella traccia
tabella_traccia

,dimensione,unità,tipo,formato,lingua,relazione
0,data,NaN,date,NaN,NaN,NaN
1,ora,NaN,time,NaN,NaN,NaN
2,filiale,NaN,categorical,NaN,NaN,NaN
3,CAP,NaN,integer categorical,NaN,NaN,NaN
4,regione,NaN,categorical,NaN,NaN,NaN
5,codice prodotto,NaN,integer categorical,NaN,NaN,NaN
6,prodotto,NaN,categorical,NaN,NaN,NaN
7,classe prodotto,NaN,categorical,NaN,NaN,NaN
8,famiglia prodotto,NaN,categorical,NaN,NaN,NaN
9,operatore,NaN,categorical,NaN,NaN,NaN


In [838]:
""""
Rimozione unità di misura


units_columns = tabella_traccia.loc[tabella_traccia['unità'].notna()]

dim_names = units_columns['dimensione']

for dim in dim_names:
    df_fact_table[dim].apply(lambda x: parserExtractQuantity(x))
    #dim_type = date_columns.apply(lambda x: x['tipo'], axis=1)
"""

'"\nRimozione unità di misura\n\n\nunits_columns = tabella_traccia.loc[tabella_traccia[\'unità\'].notna()]\n\ndim_names = units_columns[\'dimensione\']\n\nfor dim in dim_names:\n    df_fact_table[dim].apply(lambda x: parserExtractQuantity(x))\n    #dim_type = date_columns.apply(lambda x: x[\'tipo\'], axis=1)\n'

In [839]:
# Controllo visivo dopo l'individuazione dei tipi
df_fact_table[tabelle_esterne.keys()]

,data,ora,filiale,CAP,regione,codice prodotto,prodotto,classe prodotto,famiglia prodotto,operatore,operazione,unità,importo,COD
0,15/12/2021,08:30,Torino,10121,Piemonte,123,Kiwi Phone 12 Pro,Smartphone,Personal Devices,Camillo Benso,Vendita,2.0,"1.980,00 €",123
1,15/12/2021,09:30,Genova,16167,Liguria,123,Kiwi Phone 12 Pro,Smartphone,Personal Devices,Gino Paoli,Vendita,1.0,"975,59 €",123
2,15/12/2021,10:30,Potenza,85100,Basilicata,456,Bob Kitchen Robot,Elettrodomestici,Home Devices,Alberto Lupo,Vendita,1.0,"2.547,00 €",456
3,15/12/2021,11:30,Verona,37125,Veneto,456,Bob Kitchen Robot,Elettrodomestici,Home Devices,Billy Ballo,Vendita,1.0,"2.869,79 €",456
4,16/12/2021,12:30,Torino,10121,Piemonte,789,Carl Vacuum Cleaner,Elettrodomestici,Home Devices,Camillo Benso,Riparazione,NaN,NaN,789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,07/05/2022,21:45,Torino,10121,Piemonte,105,Jack Earphones 4,Audio,Personal Devices,Vittorio Emanuele,Vendita,1.0,"92,95 €",105
196,08/05/2022,10:30,Roma,135,Lazio,911,Mango Pad 8 Mini,Tablet,Personal Devices,Anco Marzio,Vendita,2.0,"1.211,60 €",911
197,08/05/2022,11:30,Torino,10121,Piemonte,456,Bob Kitchen Robot,Elettrodomestici,Home Devices,Vittorio Emanuele,Vendita,1.0,"2.499,90 €",456
198,09/05/2022,12:30,Asti,14100,Piemonte,774,Mango Pad 8 Pro Max,Tablet,Personal Devices,Pino Loricato,Vendita,1.0,"1.029,60 €",774


### ➔ Eliminazione delle tabelle esterne di tipo float, string (e integer non categoriche)

Elimino le tabelle esterne di tipo float, string e int non categorichi. `Per le tabelle esterne ti tipo integer devo ancora individuare in modo furbo per distinguere quelle categoriche da quelle non caegoriche.`

In [840]:
# Rimuovo le tabelle eterne che sono di tipo float dal dizionazio tabelle_esterne

for row in tabella_traccia.values:
    if row[2] == 'float':
        tabelle_esterne.pop(row[0], None)
        print('Tabella esterna di tipo float rimossa: ', row[0])
    #elif row[2] == 'string':
     #   tabelle_esterne.pop(row[0], None)
      #  print('Tabella esterna di tipo string rimossa: ', row[0])

Tabella esterna di tipo float rimossa:  importo


# `Individuazione delle gerarchie e delle sinonimie`

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last"><strong>Questa operazione va eseguita solamente sulle colonne di tipo categorico</strong> (includendo quindi anche le colonne integer categoriche).
</p>

Considerando le tabelle esterne su cui si è già applicata la `distinct`, prendendo in considerazione le tabelle esterne a coppie. Ogni tabella va confrontata con le altre, una sola volta e senza che l’ordine abbia importanza.

Bisogna confrontare la numerosità della distinct della tabella esterna A, la numerosità della distinct della tabella esterna B e la numerosità della distinct delle tabelle A e B insieme.
A questo punto si possono verificare le seguenti situazioni:

Ho una `GERARCHIA`. La colonna B è un genitore di A, se ad esempio:

    molteplicità Distinct(A) = 12
    molteplicità Distinct(B) = 4
    molteplicità Distinct(AB) = 12

Le tabelle esterne A e B rappresentano una `SINONIMIA`, se ad esempio:

    molteplicità Distinct(A) = 8
    molteplicità Distinct(B) = 8
    molteplicità Distinct(AB) = 8

`Non c’è invece alcuna relazione` tra le colonne A e B, se ad esempio:

    molteplicità Distinct(A) = 13
    molteplicità Distinct(B) = 28
    molteplicità Distinct(AB) = 7

<div class="admonition warning alert alert-danger">
<p class="first admonition-title" style="font-weight: bold;">Attenzione</p>
<p class="last">
Una volta stabilite le <strong>relazioni gerarchiche</strong> tra le colonne bisogna attribuire le giuste relazioni gerarchiche<strong>, mantenendo solo quelle dirette</strong> (in quanto anche i parenti di livello superiore risulteranno in relazione 1 a n ma non vanno considerati), come nel caso di mesi, trimestri e semestri.
</p>
<p class="last">   
Bisogna, inoltre, <strong>raggruppare eventuali sinonimie</strong> relative a coppie collegate, in tuple di più di due colonne.
</p>
<p>
Possono essere presenti più sinonimi e più <strong>gerarchie parallele</strong>.
</p>
</div>

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">
Giunti a questo punto, le tabelle esterne relative alle colonne di livello gerarchico più basso (figli) rappresentano le <strong>tabelle elementari del modello Luna</strong>.
</p>
</div>

In [841]:
"""
get_relation(<DataFrame>, <string>, <string>): funzione che stabilisce la relazione che intercorre tra due colonne della fact table
params:
    <DataFrame>: DataFrame, la fact table con colonne selezionate
    <string>: string le dimensioni di cui voglio sapere la relazione
output:
    String
"""

def get_relation(df, dimA, dimB):
    
    distinctA = len(tabelle_esterne[dimA].index)
    distinctB = len(tabelle_esterne[dimB].index)
    distinctAB = len(df[[dimA,dimB]].dropna().drop_duplicates().index)
   
    if distinctA == distinctB == distinctAB:
        return 'sinonimia'

    elif distinctA == distinctAB and distinctB != distinctAB:
        return 'many to one'

    elif distinctB == distinctAB and distinctA != distinctAB:
        return 'one to many'
    else:
        return 'None'

Per studiare le relazioni di gerarchichia/similitudine tra le colonne categoriche della fact table, bisogna `costruire` una sorta di **matrice di correlazione**, che nell'algoritmo viene salvata nella varibile **df_correlazioni** sottoforma di Pandas DataFrame. `Tale matrice risulterà fondamentale nelle fasi successive dell'algoritmo.`

In [842]:
# Algoritmo più oneroso dal punto di vista computazionale, ma permette di impiegare algoritmi più semplici in seguito.

# Considero solamente le colonne selezionate come categoriche
df_relazioni = df_fact_table[tabelle_esterne.keys()]

init_columns = np.array(df_relazioni.columns)
len_init_columns = len(init_columns)

relazioni = {}

for column in range(0,len_init_columns):
    
    b = []
    for dim in range(0,len_init_columns):
        b.append(get_relation(df_relazioni, init_columns[column], init_columns[dim]))
        
    relazioni[init_columns[column]] = b

    
df_correlazioni = pd.DataFrame(relazioni)
df_correlazioni = df_correlazioni.set_index(df_relazioni.columns[0:])

In [843]:
# In questa matrice sono presenti TUTTE le sinonimie e le gerarchie del modello
df_correlazioni

,data,ora,filiale,CAP,regione,codice prodotto,prodotto,classe prodotto,famiglia prodotto,operatore,operazione,unità,COD
data,sinonimia,None,None,None,None,None,None,None,None,None,None,None,None
ora,None,sinonimia,None,None,None,None,None,None,None,None,None,None,None
filiale,None,None,sinonimia,sinonimia,one to many,None,None,None,None,many to one,None,None,None
CAP,None,None,sinonimia,sinonimia,one to many,None,None,None,None,many to one,None,None,None
regione,None,None,many to one,many to one,sinonimia,None,None,None,None,many to one,None,None,None
codice prodotto,None,None,None,None,None,sinonimia,sinonimia,one to many,one to many,None,None,None,sinonimia
prodotto,None,None,None,None,None,sinonimia,sinonimia,one to many,one to many,None,None,None,sinonimia
classe prodotto,None,None,None,None,None,many to one,many to one,sinonimia,one to many,None,None,None,many to one
famiglia prodotto,None,None,None,None,None,many to one,many to one,many to one,sinonimia,None,None,None,many to one
operatore,None,None,one to many,one to many,one to many,None,None,None,None,sinonimia,None,None,None


<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">
La matrice presentata qui sopra è una "finta" <strong>matrice delle correlazioni</strong>. Non sempre i valori restituiti dalle <strong>distinct</strong> tornano, a volte sono leggermente discostati per via della mancanza di integrità nei dati forniti all'algoritmo. Se i dati forniti derivano da un database relazionale, questo problema non dovrebbe sussistere. Altrimenti si potrebbe pensare di assegnare un valore a ciascuna relazione ad andare a studiare le relazioni tramite una "vera" matrice delle correlazioni. <strong>Questo permetterebbe di recuperare gerarchie che verrebbero scartate a causa di un piccolissimo scostamento dei valori delle distinct.</strong>
</p>
</div>

### `Output Atteso`

L’output dell'algoritmo successivo corrisponde `all’aggiunta di colonne alla tabella esterna nel caso in cui siano state individuate gerarchie e sinonimie`. Per esempio:

<img src="./figure/tabella_esterna_finale.jpg" alt="Drawing" style="width: 70%; margin-top: 20px; margin-bottom: 20px"/>

* Se è stata individuata una **sinonimia** si `aggiunge una colonna dedicata con il valore sinonimo`.

* Se è stata individuata una **gerarchia** si `aggiunge invece una colonna che conterrà gli id relativi ai genitori nelle rispettive tabelle esterne`.

### `Esempio Completo`

`Nell’esempio seguente è presente un classico esempio di tabella base che presenta sia sinonimie che gerarchie.`
CODICE PRODOTTO rappresenta una sinonimia del prodotto stesso, CATEGORIA rappresenta un livello gerarchico di prodotto e FAMIGLIA rappresenta un livello gerarchico di CATEGORIA (e quindi anche di prodotto). La <strong>struttura finale delle tabelle eterne</strong> risulta pertanto essere:

* Tabella esterna “Prodotto”
<img src="./figure/tabella_prodotto.jpg" alt="Drawing" style="width: 65%; margin-bottom: 20px"/>

* Tabella esterna “Categoria” (rappresenta il livello “Padre”)
<img src="./figure/tabella_categoria.jpg" alt="Drawing" style="width: 55%; margin-bottom: 20px"/>

* Tabella esterna “Famiglia” (rappresenta il livello “Nonno”)
<img src="./figure/tabella_famiglia.jpg" alt="Drawing" style="width: 38%; margin-bottom: 20px"/>

Ovviamente, la gerarchia è presente anche tra Prodotto e Famiglia, tuttavia è "annegata” nel link ai rispettivi indici.
L’algoritmo di individuazione delle gerarchie restituirà anche la relazione gerarchica Prodotto-Famiglia, tuttavia, `nelle tabelle esterne, bisogna essere in grado di mantenere solamente un legame tra i livelli gerarchici diretti tramite gli indici.`

### ➔ Sinonimie
Innanzitutto, isolo dalla `matrice delle relazioni` tutte le <strong>sinonimie</strong>. Associo tra di loro le sinonimie, dopodiché elimino dalla matrice tutte le dimensioni legate da sinonimie tranne una, la quale mi servirà per trattare successivamente le `gerarchie` e sarà al tempo stesso di riferimento per le sinonimie trovate.

In [844]:
for dim in df_correlazioni:
    try:
        df_sinonimia = df_correlazioni.loc[df_correlazioni[dim] == 'sinonimia', df_correlazioni[dim] == 'sinonimia']
        
        if len(df_sinonimia.index) > 1:
            # Sostituisco distinct(AB) alle tab esterne di A e B ed elimino la colonna A o B elimino da df_correlazioni
            # Considero il caso più generico in cui si abbiano più di due sinonimie
            tabelle_esterne[df_sinonimia.columns[0]] = df_fact_table[df_sinonimia.columns].dropna().drop_duplicates().reset_index(drop=True)
            for k in df_sinonimia.columns[1:].tolist():
                # Flaggo nella tabella traccia le dimensioni che sono state inglobate in altre tabelle esterne per la relazione di similitudine 
                tabella_traccia.loc[tabella_traccia['dimensione'] == k, 'relazione'] = 'similitudine'
                # Rimuovo la dimensione il relazione di similitudine dal diszionario delle tabelle esterne
                tabelle_esterne.pop(k, None)
            df_correlazioni = df_correlazioni.drop(index=df_sinonimia.columns[1:].tolist(), columns=df_sinonimia.columns[1:].tolist())
    except:
        pass

# Questo algoritmo è da efficientare!

In [845]:
# Test sulle sinonimie trovate
#tabelle_esterne['mese']

### ➔ Gerarchie
Isolo dalla `matrice delle correlazioni` tutte e solo le <strong>gerarchie</strong>.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">Risulta fondamentale <strong>individuare per prima le foglie della gerarchia</strong>. In questo modo posso risalire le gerarchie passo a passo attraverso tutte le ramificazioni delle possibili gerarchie parallele.
</p>

In [846]:
df_gerarchie = df_correlazioni.loc[[((df_correlazioni.loc[index] == 'one to many').any() or (df_correlazioni.loc[index] == 'many to one').any()) for index in df_correlazioni],[((df_correlazioni[col] == 'many to one').any() or (df_correlazioni[col] == 'one to many').any()) for col in df_correlazioni.columns]]

In [847]:
# In questa matrice sono presenti TUTTE e SOLO le gerarchie del modello
df_gerarchie

,filiale,regione,codice prodotto,classe prodotto,famiglia prodotto,operatore
filiale,sinonimia,one to many,None,None,None,many to one
regione,many to one,sinonimia,None,None,None,many to one
codice prodotto,None,None,sinonimia,one to many,one to many,None
classe prodotto,None,None,many to one,sinonimia,one to many,None
famiglia prodotto,None,None,many to one,many to one,sinonimia,None
operatore,one to many,one to many,None,None,None,sinonimia


In [848]:
"""
sort_gerarchia(<dictionary>, <list>): funzione che ordina le relazioni gerarchiche sulla base della molteplicità della distinct
params:
    <dictionary>: dictionay, il dizionario contenente le tabelle esterne
    <list>: list, lista contentente in ordine sparso il nome delle colonne in relazione gerarchica
output:
    <list>: list, lista contentente il nome delle colonne in relazione gerarchica in modo ordinato
"""

def sort_gerarchia(tab, list_gerarchia):
    multiplicity = [len(tab.get(key).index) for key in list_gerarchia]
    gerarchia_sorted = [x for _, x in sorted(zip(multiplicity, list_gerarchia), reverse=True)]
    
    return gerarchia_sorted

In [849]:
"""
scan_gerarchia(<dictionary>, <DataFrame>): funzione che restituisce TUTTE le cerarchie presenti sotto forma di list di list ordinate
params:
    <dictionary>: dictionay, il dizionario contenente le tabelle esterne
    <DataFrame>: DataFrame, la fact table originaria
output:
    <list>: list, lista contentente tutte le gerarchie presenti in modo ordinato
"""

def scan_gerarchia(tab, df):
    
    # Definisco la lista in cui salvare tutte le gerarchia parallele che trovo
    gerarchie_parallele = []
    
    
    # Loop su tutte le relazioni gerarchiche
    for column in df:

        # Cerco tutte le eventuali gerarchie parallele presenti
        # Individuo i livelli più alti di ciascuna gerarchia parallela con un criterio di selezione sulle colonne del dataFrame
        if 'many to one' not in df[column].values:

            # Seleziono tutte le dimensioni in relazione gerarchica al dato livello gerarchico superiore
            gerarchia_parallela = df.index[df[column] == 'one to many'].tolist()

            gerarchia_parallela = sort_gerarchia(tab, gerarchia_parallela)

            leaf_dim = gerarchia_parallela[0]
            root_dim = column
            gerarchia_parallela = gerarchia_parallela[1:]
            
            if len(gerarchia_parallela) > 1:
                df_ = df.loc[gerarchia_parallela, gerarchia_parallela]

                if ('None' in df_.values):
                    gerarchia_parallela_ = []
                    for i in range(1, len(df_.index)):
                        gerarchia_parallela__ = [df_.index[i]]

                        for j in range(0, i):
                            if(df_.iloc[i, j] !='None'):
                                gerarchia_parallela__.append(df_.columns[j])
                                
                        if len(gerarchia_parallela__) > 1:
                            gerarchia_parallela__ = sort_gerarchia(tab, gerarchia_parallela__)
                            gerarchia_parallela__.insert(0,leaf_dim)
                            gerarchia_parallela__.append(root_dim)
                            gerarchia_parallela_.append(gerarchia_parallela__)
                            gerarchie_parallele.append(gerarchia_parallela__)
                        else:
                            continue
                            
                    # Aggiungo anche tutte le gerarchie singole, che sono in relazione "none" con tutte le restanti parallele
                    gerarchia_parallela_list = [item for sublist in gerarchia_parallela_ for item in sublist]
                    for col in df_.columns:
                        if col not in gerarchia_parallela_list:
                            col_ = [col]
                            col_.insert(0,leaf_dim)
                            col_.append(root_dim)
                            gerarchie_parallele.append(col_)
                        else:
                            continue
                else:
                    gerarchia_parallela.insert(0,leaf_dim)
                    gerarchia_parallela.append(root_dim)
                    gerarchie_parallele.append(gerarchia_parallela)
            else:       
                gerarchia_parallela.insert(0,leaf_dim)
                gerarchia_parallela.append(root_dim)
                gerarchie_parallele.append(gerarchia_parallela)
        else:
            continue

    gerarchie_parallele_set = set(tuple(x) for x in gerarchie_parallele)
    gerarchie_parallele = [list(x) for x in gerarchie_parallele_set]
    
    ger_par_final = [] 
    
    for ger in gerarchie_parallele:
        ger_ = ger[1:-1]
        
        ger_spurie = []
        for i in range(0, len(ger_)):
            for j in range(i+1,len(ger_)):
                if(df.loc[ger_[i],ger_[j]] == 'None'):
                    ger_spurie.append(ger_[i])
                    ger_spurie.append(ger_[j])
                else:
                    continue

        if len(ger_spurie) == 0:
            ger_par_final.append(ger)
        else:                
            a = set(ger_spurie) ^ set(ger)
            for coll1 in ger_spurie:
                b = list(a)
                for coll2 in ger_spurie:
                    if(df.loc[coll1,coll2] != 'None'):
                        b.append(coll1)
                        if coll1 != coll2:
                            b.append(coll2)
                b = sort_gerarchia(tab, b)
                ger_par_final.append(b)
                
    gerarchie_parallele_set = set(tuple(x) for x in ger_par_final)
    gerarchie_parallele = [list(x) for x in gerarchie_parallele_set]
            
    return gerarchie_parallele

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota sull implementazione dell'algorimo che decodifica le gerarchie (anche parallele)</p>
<p class="last">
L'idea di base per <strong>trovare tutte le gerarchie parallele</strong> è quella di:</p>
<ul>
<li>Individuo i livelli più alti di ciascuna gerarchia parallela con un criterio di selezione sulle colonne dell fact table</li>
<li>Seleziono tutte le dimensioni in relazione gerarchica al livello gerarchico superiore</li>
<li>Le ordino in base alla molteplicità ottenuta con le distinct</li>
<li>Data la foglia delle gerarchie parallele, le risalgo tutte fino ai "capostipiti".</li>
</ul>

In [850]:
# Test sulle gerarchie paralle torvate
scan_gerarchia(tabelle_esterne,df_gerarchie)

[['operatore', 'filiale', 'regione'],
 ['codice prodotto', 'classe prodotto', 'famiglia prodotto']]

In [851]:
# Ridefinizione della funzioni originale per testare per l'algoritmo che identifica le gerarchie
def sort_gerarchia(tabelle_esterne, gerarchia):
    multiplicity = [tabelle_esterne[key] for key in gerarchia]
    gerarchia_sorted = [x for _, x in sorted(zip(multiplicity, gerarchia), reverse=True)]
    
    return gerarchia_sorted

In [852]:
# initialize list of lists
test_data1 = {'A':['sinonimia','many to one','many to one','many to one','many to one','many to one','many to one','many to one','many to one','many to one','many to one'],
        'B':['one to many','sinonimia','None','None','None','None','None','None','None','None','None'],
        'C':['one to many','None','sinonimia','None','many to one','None','None','many to one','None','None','None'],
        'D':['one to many','None','None','sinonimia','None','many to one','many to one','many to one','many to one','many to one','many to one'],
        'E':['one to many','None','one to many','None','sinonimia','None','None','many to one','None','None','None'],
        'F':['one to many','None','None','one to many','None','sinonimia','None','many to one','many to one','None','None'],
        'G':['one to many','None','None','one to many','None','None','sinonimia','None','None','many to one','many to one'],
        'H':['one to many','None','one to many','one to many','one to many','one to many','None','sinonimia','None','None','None'],
        'I':['one to many','None','None','one to many','None','one to many','None','None','sinonimia','None','None'],
        'L':['one to many','None','None','one to many','None','None','one to many','None','None','sinonimia','many to one'],
        'M':['one to many','None','None','one to many','None','None','one to many','None','None','one to many','sinonimia']}

tabelle_esterne_data1 = {'A': 100,'B':50,'C':60,'D':40,'E':30,'F':30,'G':35,'H':20,'I':10,'L':20,'M':5}

df_data1 = pd.DataFrame(test_data1, index=['A','B','C','D','E','F','G','H','I','L','M'])
df_data1

,A,B,C,D,E,F,G,H,I,L,M
A,sinonimia,one to many,one to many,one to many,one to many,one to many,one to many,one to many,one to many,one to many,one to many
B,many to one,sinonimia,None,None,None,None,None,None,None,None,None
C,many to one,None,sinonimia,None,one to many,None,None,one to many,None,None,None
D,many to one,None,None,sinonimia,None,one to many,one to many,one to many,one to many,one to many,one to many
E,many to one,None,many to one,None,sinonimia,None,None,one to many,None,None,None
F,many to one,None,None,many to one,None,sinonimia,None,one to many,one to many,None,None
G,many to one,None,None,many to one,None,None,sinonimia,None,None,one to many,one to many
H,many to one,None,many to one,many to one,many to one,many to one,None,sinonimia,None,None,None
I,many to one,None,None,many to one,None,many to one,None,None,sinonimia,None,None
L,many to one,None,None,many to one,None,None,many to one,None,None,sinonimia,one to many


La variabile test_data1 rappresenta la seguente gerarchia complessa:

<img src="./figure/gerarchia_test1.png" style="width: 50%; margin-bottom: 20px"/>

Le `gerarchi parallele` che l'algoritmo deve individuare sono:

* `['A', 'D', 'F', 'H']`
* `['A', 'D', 'G', 'L', 'M']`
* `['A', 'C', 'E', 'H']`
* `['A', 'B']`
* `['A', 'D', 'F', 'I']`

In [853]:
scan_gerarchia(tabelle_esterne_data1, df_data1)

[['A', 'D', 'F', 'H'],
 ['A', 'D', 'G', 'L', 'M'],
 ['A', 'D', 'F', 'I'],
 ['A', 'B'],
 ['A', 'C', 'E', 'H']]

In [854]:
# initialize list of lists
test_data2 = {'A':['sinonimia','many to one','many to one','many to one','many to one','many to one'],
              'B':['one to many','sinonimia','None','None','None','many to one'],
              'C':['one to many','None','sinonimia','None','None','many to one'],
              'D':['one to many','None','None','similitudine','None','many to one'],
              'E':['one to many','None','None','None','similitudine','None'],
              'F':['one to many','one to many','one to many','one to many','None','similitudine']}

tabelle_esterne_data2 = {'A': 100,'B':50,'C':60,'D':40,'E':30,'F':5}

df_data2 = pd.DataFrame(test_data2, index=['A','B','C','D','E','F'])
#df_data2

La variabile test_data2 rappresenta la seguente gerarchia complessa:

<img src="./figure/gerarchia_test2.png" style="width: 50%; margin-bottom: 20px"/>

Le `gerarchi parallele` che l'algoritmo deve individuare sono:

* `['A', 'B', 'F']`
* `['A', 'C', 'F']`
* `['A', 'D', 'F']`
* `['A', 'E']`

In [855]:
scan_gerarchia(tabelle_esterne_data2, df_data2)

[['A', 'B', 'F'], ['A', 'C', 'F'], ['A', 'D', 'F'], ['A', 'E']]

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">
Continuare a creare gerarchie strane e complesse per testare il funzionamento dell'algoritmo <strong>scan_gerarchia</strong>. </p>
    <p><strong>P.S.</strong> Al momento è in grado di  gestire tutti i casi considerati correttamente :)</p>
</div>

In [856]:
# DUPLICATO: Overwriting the sort_gerarchia function again to use the original and not testing definition
def sort_gerarchia(tabelle_esterne, list_gerarchia):
    multiplicity = [len(tabelle_esterne.get(key).index) for key in list_gerarchia]
    gerarchia_sorted = [x for _, x in sorted(zip(multiplicity, list_gerarchia), reverse=True)]
    
    return gerarchia_sorted

Ritornando al nostro algoritmo, eseguo uno scan delle gerarchie, che `mi restituisce una lista contenente tutte le gerarchie parallele presenti nella fact table.`

In [857]:
gerarchie_parallele = scan_gerarchia(tabelle_esterne, df_gerarchie)
gerarchie_parallele

[['operatore', 'filiale', 'regione'],
 ['codice prodotto', 'classe prodotto', 'famiglia prodotto']]

In [858]:
"""
setChiaveEsterna(<DataFrame>, <list>): funzione che aggiunge alla tabelle esterne dei figli una colonna che contiene il corrispondente indice dei padri
params:
    <DataFrame>: DataFrame, la fact table originaria
    <list>: list, lista che contiene una gerarchia parallela già ordinata
output:
    <Bool>
"""

def setChiaveEsterna(df, gerarchia_parallela):
    for i in range(0,len(gerarchia_parallela)-1):        
        indexes = []
        dim_low = gerarchia_parallela[i]
        dim_high = gerarchia_parallela[i+1]
        
        # Flaggo nella tabella traccia le dimensioni che fanno parte di gerarchie, ma che NON sono foglie di tale gerarchia
        tabella_traccia.loc[tabella_traccia['dimensione'] == dim_high, 'relazione'] = 'gerarchia'
            
        for j in range(0,len(tabelle_esterne[dim_low])):
            valore = df.loc[df[dim_low] == tabelle_esterne[dim_low].loc[j].values[0], dim_high].dropna().drop_duplicates().values[0]
            print(dim_high)
            print(valore)
            _id = tabelle_esterne[dim_high].index[tabelle_esterne[dim_high][dim_high] == str(valore)].tolist()[0]
            indexes.append(_id)

        tabelle_esterne[dim_low].loc[:,dim_high] = indexes
    return True

Per ciascuna gerarchia (parallela) trovata, `utilizzo la lista ordinata della relazione gerarchica per andare ad aggiungere alla tabelle esterne di un figlio una colonna che contiene il corrispondente indice del padre.`

In [859]:
for i in range(0, len(gerarchie_parallele)):
    setChiaveEsterna(df_fact_table, gerarchie_parallele[i])

filiale
Torino
filiale
Genova
filiale
Potenza
filiale
Verona
filiale
Genova
filiale
Torino
filiale
Asti
filiale
Matera
filiale
Verona
filiale
Verona
filiale
Torino
filiale
Potenza
filiale
Roma
filiale
Asti
filiale
Roma
regione
Piemonte
regione
Liguria
regione
Basilicata
regione
Veneto
regione
Piemonte
regione
Basilicata
regione
Lazio
classe prodotto
Smartphone
classe prodotto
Elettrodomestici
classe prodotto
Elettrodomestici
classe prodotto
Audio
classe prodotto
Smartphone
classe prodotto
Tablet
classe prodotto
Health Device
classe prodotto
Health Device
classe prodotto
Audio
classe prodotto
Tablet
famiglia prodotto
Personal Devices
famiglia prodotto
Home Devices
famiglia prodotto
Personal Devices
famiglia prodotto
Personal Devices
famiglia prodotto
Personal Devices


In [860]:
# Esempio di tabella esterna con in cui è presente una similitudine e un riferimento al padre di una gerarchia
#tabelle_esterne['anno']

# `Output finale: "Fact Table Ristrutturata"`

L'output finale del **modulo Ingestor** è rappresentato dalla costruzione di una `“fact table
ristrutturata” ricca di metadati.`
La tabella viene generata a partire dal `file originale` (non dal campione) e secondo questi
criteri:

* <strong>le colonne base sono sostituite dagli ID delle tabelle esterne associate</strong>,

* <strong>vengono eliminate le colonne relative alle sinonimie</strong> (che sono già sulla tabella esterna associata),

* <strong>le colonne di gerarchia sono eliminate</strong> (in quanto presenti sulle tabelle esterne nella forma di snowflake),

* <strong>le colonne non categoriche sono identificate come misure</strong>. Ciascuna con il proprio tipo e con un formato di default Luna scelto da noi (tenendo però traccia del formato originale).

`La struttura finale sarà quindi a tutti gli effetti una fact table complessa di tipo snowflake.`

La fact table ristrutturata, inoltre, è etichettata con la data di analisi, il file originale con
l’indicazione dell’utente che lo ha fornito, un ID univoco e una descrizione fornita dall’utente.

<img src="./figure/fact_table_ristrutturata.jpg" alt="Drawing" style="width: 75%; margin-top: 40px; margin-bottom: 30px"/>

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip di implementazione</p>
<p class="last">
L'algoritmo deve andare a leggere dal file originale il valore di ciascuna colonna individuata
come base, cercare il valore nella rispettiva tabella esterna, individuare l’id corrispondente
ed inserirlo nella fact table ristrutturata nella giusta posizione. A questo proposito, la fact
table ristrutturata deve presentare la stessa indicizzazione della fact table originale.
La fact table ristrutturata può essere un nuovo csv oppure una vera e propria tabella di
database.
</p>
</div>


<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p class="last">
<strong>Per sfruttare meglio la memoria</strong> si è deciso di ottenere la fact table ristrutturata andando a modificare la factable originale, la quale può comunque sempre essere ricaricata su memeria dovesse essercene la necessità.
</p>

**Elimino** innanzitutto dalla fact table tutte le colonne già presenti in qualche altra tabella esterna per via di relazioni di similitudine o di gerarchia. `Sono escluse da questa rimozione tutte le foglie di una gerarchia.`

La fact table ristrutturata e tutte le informazione aggiuntive che la riguardano sono poi salvate in nel dizionario **fact_table_ristrutturata**.

In [861]:
dim_in_rel = tabella_traccia.loc[tabella_traccia['relazione'].isin(['gerarchia','similitudine']), 'dimensione'].tolist()

# Elimino dalla fact table originale tutte le dimensione in relazione gerarchica o di similitudine
for dim in dim_in_rel:
    df_fact_table = df_fact_table.drop(dim, axis=1)

In [862]:
fact_table_ristrutturata = {}
fact_table_ristrutturata['utente'] = 'Davide Marietti'
fact_table_ristrutturata['data'] = '20/01/2023'

In [863]:
# Ispezione visiva della fact table originale modificata
df_fact_table_prep = df_fact_table.copy()

Il passo finale per ottenere la factable ristrutturata è stato quello di `sostituire i dati della fact table originale con i corrispondenti indici dei valori presenti nelle tabelle esterne.`

In [864]:
# Considero SOLO tutte le tabelle esterne di dimensioni che non sono in relazione gerarchica con altre dimensioni, o che sono foglie di gerarchie (parallele)

for dim in tabelle_esterne.keys():
    if dim not in dim_in_rel:
        
        dict_dim = tabelle_esterne[dim][dim].astype('str').to_dict()
        dict_dim_res = dict((v,k) for k,v in dict_dim.items())
        
        # Devo aggiungere anche il caso in cui oh NaN, che non è contemplato nelle tabelle esterne, mentre nella fact table sì
        #dict_dim_res['NaN'] = 'NaN'

        df_fact_table_prep[dim] = df_fact_table_prep[dim].astype('str')
        df_fact_table_prep[dim] = df_fact_table_prep[dim].map(dict_dim_res, na_action='ignore').map("{:.0f}".format)
        

In [865]:
""""
Costruzione timestamp luna
"""

date_columns = tabella_traccia.loc[tabella_traccia['tipo'].isin(['date', 'time', 'year', 'month', 'timestamp'])]

dim_name = date_columns['dimensione']
dim_type = date_columns.apply(lambda x: x['tipo'], axis=1)

df_fact_table_date = df_fact_table[dim_name]
df_fact_table_date.columns= np.array(dim_type)

In [866]:
df_fact_table_date

,date,time
0,15/12/2021,08:30
1,15/12/2021,09:30
2,15/12/2021,10:30
3,15/12/2021,11:30
4,16/12/2021,12:30
...,...,...
195,07/05/2022,21:45
196,08/05/2022,10:30
197,08/05/2022,11:30
198,09/05/2022,12:30


In [867]:
buildLunaTimestamp(df_fact_table_date)
convertToLunaTimestamp(df_fact_table_date)

C:\Users\david\AppData\Local\Temp\ipykernel_8236\2802943593.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestampLuna'] = standard_timestamp('0')
C:\Users\david\AppData\Local\Temp\ipykernel_8236\2802943593.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestampLuna'] = df['date'].apply(lambda x: standard_timestamp(x))
C:\Users\david\AppData\Local\Temp\ipykernel_8236\2802943593.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,date,time,timestampLuna
0,15/12/2021,"CustomParser(0, 0, 0, 8, 30, 0, 0)",20211215083000
1,15/12/2021,"CustomParser(0, 0, 0, 9, 30, 0, 0)",20211215093000
2,15/12/2021,"CustomParser(0, 0, 0, 10, 30, 0, 0)",20211215103000
3,15/12/2021,"CustomParser(0, 0, 0, 11, 30, 0, 0)",20211215113000
4,16/12/2021,"CustomParser(0, 0, 0, 12, 30, 0, 0)",20211216123000
...,...,...,...
195,07/05/2022,"CustomParser(0, 0, 0, 21, 45, 0, 0)",20220705214500
196,08/05/2022,"CustomParser(0, 0, 0, 10, 30, 0, 0)",20220805103000
197,08/05/2022,"CustomParser(0, 0, 0, 11, 30, 0, 0)",20220805113000
198,09/05/2022,"CustomParser(0, 0, 0, 12, 30, 0, 0)",20220905123000


In [868]:
df_fact_table_prep['timestampLuna'] = df_fact_table_date['timestampLuna']
tabelle_esterne['timestampLuna'] = pd.DataFrame(df_fact_table_date['timestampLuna'])

In [869]:
fact_table_ristrutturata['fact table'] = df_fact_table_prep

In [870]:
fact_table_ristrutturata['utente']

'Davide Marietti'

In [871]:
fact_table_ristrutturata['data']

'20/01/2023'

In [872]:
fact_table_ristrutturata['fact table']

,data,ora,codice prodotto,operatore,operazione,unità,importo,timestampLuna
0,0,0,0,0,0,0,"1.980,00 €",20211215083000
1,0,1,0,1,0,1,"975,59 €",20211215093000
2,0,2,1,2,0,1,"2.547,00 €",20211215103000
3,0,3,1,3,0,1,"2.869,79 €",20211215113000
4,1,4,2,0,1,nan,NaN,20211216123000
...,...,...,...,...,...,...,...,...
195,88,12,3,5,0,1,"92,95 €",20220705214500
196,89,2,9,14,0,0,"1.211,60 €",20220805103000
197,89,3,1,5,0,1,"2.499,90 €",20220805113000
198,90,4,5,13,0,1,"1.029,60 €",20220905123000


In [873]:
tabelle_esterne['timestampLuna']

,timestampLuna
0,20211215083000
1,20211215093000
2,20211215103000
3,20211215113000
4,20211216123000
...,...
195,20220705214500
196,20220805103000
197,20220805113000
198,20220905123000


In [874]:
# TODO
# - riutilizzare le valiabili non utili andandole a sovrascriverle tramite ad esempio _

In [875]:
tabella_traccia

,dimensione,unità,tipo,formato,lingua,relazione
0,data,NaN,date,NaN,NaN,NaN
1,ora,NaN,time,NaN,NaN,NaN
2,filiale,NaN,categorical,NaN,NaN,gerarchia
3,CAP,NaN,integer categorical,NaN,NaN,similitudine
4,regione,NaN,categorical,NaN,NaN,gerarchia
5,codice prodotto,NaN,integer categorical,NaN,NaN,NaN
6,prodotto,NaN,categorical,NaN,NaN,similitudine
7,classe prodotto,NaN,categorical,NaN,NaN,gerarchia
8,famiglia prodotto,NaN,categorical,NaN,NaN,gerarchia
9,operatore,NaN,categorical,NaN,NaN,NaN
